### Imports 

In [2]:
#%load_ext cudf.pandas
#import pandas as pd

import cudf as pd
import shap
import gc


from itertools import product
from itertools import combinations
import plotly.graph_objects as go

import os

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, ConcatDataset
import torchaudio

import random
import ast
        
import sys

    
import torch.nn.functional as F
from torcheval.metrics.functional import multiclass_confusion_matrix
from torch.optim import Adam ,SGD ,Adadelta
from torch.nn import CrossEntropyLoss

from torchvision.utils import save_image
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch
import numpy as np
import torch
import torch.nn as nn
from torch.autograd.functional import vjp
import networkx as nx



import optuna

from itertools import product
from itertools import combinations
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation ,FFMpegWriter ,PillowWriter

import dill
import numpy as np

import json
import pandas as pd
from tqdm import tqdm
import copy
import os 
import random

from collections import OrderedDict
import wandb


import random
import csv 
import time
import sys
import time
import datetime
import warnings
import traceback
import gc
from accelerate import Accelerator
import numpy as np

from pathlib import Path

from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram
from gtda.diagrams import PersistenceEntropy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from gtda.plotting import plot_point_cloud

# TDA tools
from gtda.mapper import (
    CubicalCover,
    make_mapper_pipeline,
    Projection,
    plot_static_mapper_graph,
    plot_interactive_mapper_graph,
    MapperInteractivePlotter
)

# ML tools
from sklearn import datasets
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA


from gtda.mapper.filter import Projection,Entropy,Eccentricity
from gtda.mapper.cover import CubicalCover
# scikit-learn method
# giotto-tda method
from gtda.mapper.cluster import FirstSimpleGap
import warnings
warnings.filterwarnings('ignore')
# Suppress the specific warning
warnings.filterwarnings("ignore", category=UserWarning, module='threadpoolctl')
import sys
import os
import textwrap
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
# Redirect stderr to null to suppress the exception messages
sys.stderr = open(os.devnull, 'w')
import ctypes
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Suppress low-level warnings from C code
libc = ctypes.CDLL(None)
libc.prctl(15, 0, 0, 0, 0)

CUDARuntimeError: cudaErrorCompatNotSupportedOnDevice: forward compatibility was attempted on non supported HW

In [1]:
!pip freeze > requirements.txt

### W&B login 

In [2]:
import wandb
from Double_input_transformer import CustomDataset,TransformerAE
wandb.login(key="ab631efc36e2c87f5f54d82b5cdbd6c501d5221f")

seed=271
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

import kaleido
#kaleido.get_chrome_sync()

In [3]:
#os.environ["WANDB_MODE"]= "offline"
#!wandb sync
!pip show multipers

### Lazy load helper functions

In [4]:
def batchify(lst, batch_size, batch_limit=100):
    batches = [lst[i:i + batch_size] for i in range(0, len(lst), batch_size)]
    return batches[:batch_limit]

def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
def get_plot_object(vec1, tg):
    fig, ax = plt.subplots(figsize=(10, 5))  # Create figure and axis objects
    canvas = FigureCanvas(fig)  # Create a canvas to hold the figure

    # Convert tensors to NumPy and flatten
    vec1_np = vec1.cpu().numpy().flatten()
    tg_np = tg[0].cpu().numpy().flatten()

    # Plot data
    ax.plot(vec1_np, label='Predicted', alpha=0.7)
    ax.plot(tg_np, label='Target', alpha=0.7)

    # Labels and legend
    ax.legend()
    ax.set_xlabel('Neurone Index')
    ax.set_ylabel('Neurone Value')
    ax.set_title('Plot of Predicted vs Target')

    return fig 
    

### Example of static Tokenization 

In [5]:
class EmbedderNeuronGroup(nn.Module):
    def __init__(self, d_model, seed=22):
        super().__init__()
        #print("EmbedderNeuroneGroup")
        self.neuron_l1 = nn.Linear(200, d_model) #8
        self.neuron_l2 = nn.Linear(72, d_model) #12

    def forward(self, x):
        return self.multiLinear(x)

    def multiLinear(self, v):
        #print("multi-linear method",v.shape)

        l = []

        for ndx in range(8):
            idx_start = ndx * 200
            idx_end = idx_start + 200
            l.append(self.neuron_l1(v[:,idx_start:idx_end]))

        # l2
        for ndx in range(12):
            idx_start = 200*8 + ndx * 72
            idx_end = idx_start + 72
            l.append(self.neuron_l2(v[:,idx_start:idx_end]))
        #print(len(l))
        #print(len(l[0]))
        final = torch.stack(l, dim=1)

        # print(final.shape)
        return final



### Label specific DataLoader 

In [6]:
from torchvision import datasets
IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

class ClassSpecificImageFolder(datasets.DatasetFolder):
    def __init__(
            self,
            root,
            dropped_classes=[],
            transform = None,
            target_transform = None,
            loader = datasets.folder.default_loader,
            is_valid_file = None,
    ):
        self.dropped_classes = dropped_classes
        super(ClassSpecificImageFolder, self).__init__(root, loader, IMG_EXTENSIONS if is_valid_file is None else None,
                                                       transform=transform,
                                                       target_transform=target_transform,
                                                       is_valid_file=is_valid_file)
        self.imgs = self.samples

    def find_classes(self, directory):
        classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
        classes = [c for c in classes if c not in self.dropped_classes]
        if not classes:
            raise FileNotFoundError(f"Couldn't find any class folder in {directory}.")

        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

### CNN Class

In [7]:

#from torcheval.metrics.functional import multiclass_confusion_matrix

class CNN(nn.Module):
    def __init__(
        self,
        channels_in,
        nlin="leakyrelu",
        dropout=0.0,
        init_type="uniform",
    ):
        super().__init__()
        # init module list
        self.module_list = nn.ModuleList()
        ### ASSUMES 28x28 image size
        ## compose layer 1
        self.module_list.append(nn.Conv2d(channels_in, 8, 5))
        self.module_list.append(nn.MaxPool2d(2, 2))
        self.module_list.append(self.get_nonlin(nlin))
        # apply dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## compose layer 2
        self.module_list.append(nn.Conv2d(8, 6, 5))
        self.module_list.append(nn.MaxPool2d(2, 2))
        self.module_list.append(self.get_nonlin(nlin))
        ## add dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## compose layer 3
        self.module_list.append(nn.Conv2d(6, 4, 2))
        self.module_list.append(self.get_nonlin(nlin))
        ## add flatten layer
        self.module_list.append(nn.Flatten())
        ## add linear layer 1
        self.module_list.append(nn.Linear(3 * 3 * 4, 20))
        self.module_list.append(self.get_nonlin(nlin))
        ## add dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## add linear layer 1
        self.module_list.append(nn.Linear(20, 10))

        ### initialize weights with se methods
        self.initialize_weights(init_type)

    def initialize_weights(self, init_type):
        # print("initialze model")
        for m in self.module_list:
            if type(m) == nn.Linear or type(m) == nn.Conv2d:
                if init_type == "xavier_uniform":
                    torch.nn.init.xavier_uniform_(m.weight)
                if init_type == "xavier_normal":
                    torch.nn.init.xavier_normal_(m.weight)
                if init_type == "uniform":
                    torch.nn.init.uniform_(m.weight)
                if init_type == "normal":
                    torch.nn.init.normal_(m.weight)
                if init_type == "kaiming_normal":
                    torch.nn.init.kaiming_normal_(m.weight)
                if init_type == "kaiming_uniform":
                    torch.nn.init.kaiming_uniform_(m.weight)
                # set bias to some small non-zero value
                m.bias.data.fill_(0.01)

    def get_nonlin(self, nlin):
        # apply nonlinearity
        if nlin == "leakyrelu":
            return nn.LeakyReLU()
        if nlin == "relu":
            return nn.ReLU()
        if nlin == "tanh":
            return nn.Tanh()
        if nlin == "sigmoid":
            return nn.Sigmoid()
        if nlin == "silu":
            return nn.SiLU()
        if nlin == "gelu":
            return nn.GELU()


    def forward(self, x):
        # forward prop through module_list
        for layer in self.module_list:
            x = layer(x)
        return x

    def forward_activations(self, x):
        # forward prop through module_list
        activations = []
        for layer in self.module_list:
            x = layer(x)
            if (
                isinstance(layer, nn.Tanh)
                or isinstance(layer, nn.Sigmoid)
                or isinstance(layer, nn.ReLU)
                or isinstance(layer, nn.LeakyReLU)
                or isinstance(layer, nn.SiLU)
                or isinstance(layer, nn.GELU)
                or isinstance(layer, ORU)
                or isinstance(layer, ERU)
            ):
                activations.append(x)
        return x, activations


### Training method

In [8]:
def train(model, trainloader, optimizer, criterion,nb_classes,First=False,df=None,verbose=False,log_freq_steps=25):
    List_mx=[]
    model.train()
    print(f'Training {str(datetime.datetime.now())}')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in enumerate(trainloader):
        counter += 1
        image, labels = data
        image = image
        labels = labels
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        #mx=multiclass_confusion_matrix(preds ,labels,nb_classes,normalize="pred")
        #List_mx.append(mx)
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
        if First==True and i%log_freq_steps==0 :
            epoch_loss = train_running_loss / counter
            epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
            if verbose==True:
                print(epoch_acc,"%")
            #print(f"step {i}:",epoch_loss, epoch_acc)
            if type(df)!='NoneType':
                df.at[track,f"Step {i}"]=epoch_acc
    
    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc



### Validation method

In [9]:
def validate(model, testloader, criterion, nb_classes,epoch,vec):
    model.eval()
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    correct_counts = {str(i): 0 for i in range(nb_classes+1)}
    total_counts = {str(i): 0 for i in range(nb_classes+1)}
    #shapley_values_per_class = {str(i): [] for i in range(nb_classes)}
    with torch.no_grad():
        for i, data in enumerate(testloader):
            if data is None:
                continue
            counter += 1
            image, labels = data
            outputs = model(image)
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
            # Update correct/total counts
            for j in range(len(labels)):
                class_label = str(labels[j].item())
                total_counts[class_label] += 1
                if preds[j] == labels[j]:
                    correct_counts[class_label] += 1
            for j in range(len(labels)):
                class_label = str(labels[j].item())
                #shapley_values_per_class[class_label].append(np.mean(shap_values[j].numpy()))
    
    # Compute final metrics
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))

    wandb.log({
        f"Epoch {epoch} Loss batch {vec} ": epoch_loss,
        f"Epoch{epoch}  Accuracy {vec}": epoch_acc})
    return epoch_loss, epoch_acc,correct_counts,total_counts
def create_frame(step,ax,data):
    ax=ax.cla()
    sns.heatmap(data[step][-1].cpu(),annot=True,cmap="cubehelix",ax=ax,cbar=False)
    plt.title('Epoch {} training {}'.format(step,exp)  )

### Record-Tracking Dataframes

In [10]:
Cols=["label task 1","index","Accuracy task1",\
      "label task 2","Accuracy task2"]+ \
["weight {}".format(x) for x in range(200)]+["bias {}".format(x) for x in range(200,208)]+ \
["weight {}".format(x) for x in range(208,1408)]+["bias {}".format(x) for x in range(1408,1414)]+ \
["weight {}".format(x) for x in range(1414,1510)]+["bias {}".format(x) for x in range(1510,1514)]+ \
["weight {}".format(x) for x in range(1514,2234)]+["bias {}".format(x) for x in range(2234,2254)]+ \
["weight {}".format(x) for x in range(2254,2454)]+["bias {}".format(x) for x in range(2454,2464)]+ \
["Loader Set","Reconstructed Accuracy ID","Actual Accuracy","Reconstructed Accuracy OOD","Transformer Loss","lr",'epochCNN','ActivationCNN'] 

print(len(Cols))
predicted_Weights= pd.DataFrame(columns=Cols)

# row=["".format(task1),int(ind[0]),ACC[0],"".format(task2),ACC[1]]+vector_aux.to_list()+["train",valid_epoch_acc0,ACC[2],valid_epoch_acc1,L_train[-1]]
# predicted_Weights.append(row, ignore_index=True)
predicted_Weights

2477


,label task 1,index,Accuracy task1,label task 2,Accuracy task2,weight 0,weight 1,weight 2,weight 3,weight 4,...,bias 2462,bias 2463,Loader Set,Reconstructed Accuracy ID,Actual Accuracy,Reconstructed Accuracy OOD,Transformer Loss,lr,epochCNN,ActivationCNN


In [11]:
import random
import csv 
import time
import sys
import time
import datetime
import warnings
import traceback
import gc
from accelerate import Accelerator
from scipy.stats import wasserstein_distance

Cols=["label task 1","label task 2"]+ \
["weight {}".format(x) for x in range(200)]+["bias {}".format(x) for x in range(200,208)]+ \
["weight {}".format(x) for x in range(208,1408)]+["bias {}".format(x) for x in range(1408,1414)]+ \
["weight {}".format(x) for x in range(1414,1510)]+["bias {}".format(x) for x in range(1510,1514)]+ \
["weight {}".format(x) for x in range(1514,2234)]+["bias {}".format(x) for x in range(2234,2254)]+ \
["weight {}".format(x) for x in range(2254,2454)]+["bias {}".format(x) for x in range(2454,2464)]+ \
["Pred weight {}".format(x) for x in range(200)]+["Pred bias {}".format(x) for x in range(200,208)]+ \
["Pred weight {}".format(x) for x in range(208,1408)]+["Pred bias {}".format(x) for x in range(1408,1414)]+ \
["Pred weight {}".format(x) for x in range(1414,1510)]+["Pred bias {}".format(x) for x in range(1510,1514)]+ \
["Pred weight {}".format(x) for x in range(1514,2234)]+["Pred bias {}".format(x) for x in range(2234,2254)]+ \
["Pred weight {}".format(x) for x in range(2254,2454)]+["Pred bias {}".format(x) for x in range(2454,2464)]+ \
["FN weight {}".format(x) for x in range(200)]+["FN bias {}".format(x) for x in range(200,208)]+ \
["FN weight {}".format(x) for x in range(208,1408)]+["FN bias {}".format(x) for x in range(1408,1414)]+ \
["FN weight {}".format(x) for x in range(1414,1510)]+["FN bias {}".format(x) for x in range(1510,1514)]+ \
["FN weight {}".format(x) for x in range(1514,2234)]+["FN bias {}".format(x) for x in range(2234,2254)]+ \
["FN weight {}".format(x) for x in range(2254,2454)]+["FN bias {}".format(x) for x in range(2454,2464)]+ \
["Actual Accuracy","Reconstructed Accuracy ID","Transformer train Loss"]+\
["MSE","MSE 1","MSE 2","MSE 3","MSE 4","MSE 5","KLD","KL 1","KL 2","KL 3","KL 4","KL 5","LWLN"]+\
["Wasserstein Loss","WS 1","WS 2","WS 3","WS 4","WS 5","contractive distance","N1","N11","N12","N13","N14","N15","N2","N21","N22","N23","N24","N25","saturated in pred(%)","saturated in GT(%)"]+\
["MSE FN","MSE 1 FN","MSE 2 FN","MSE 3 FN","MSE 4 FN","MSE 5 FN","KL divergence FN","KL 1 FN","KL 2 FN","KL 3 FN","KL 4 FN","KL 5 FN","LWLN FN"]+\
["WSD FN","WS 1 FN","WS 2 FN","WS 3 FN","WS 4 FN","WS 5 FN","contractive distance FN","N1 FN","N11 FN","N12 FN","N13 FN","N14 FN","N15 FN","N2 FN","N21 FN","N22 FN","N23 FN","N24 FN","N25 FN","saturated in pred FN(%)","saturated in GT FN(%)"]+\
["Step 0","Step 25","Step 50","Step 75","Step 100","Step 125","Step 150","Step 175","Step 200","Step 225","Step 250","Step 275","epoch 0","epoch 1","epoch 2"]+\
["epoch 3","epoch 4","epoch 5","epoch 6","epoch 7"]+["Representation Z{}".format(x) for x in range(256)]+["Z1 L2","Z1 L2","z1 L2","z2 L2"]
#incoporate Norm                                                     
#The Fisher information metric X norm of the target  
#dataset distance , topological distance ,UMAP distance

print(len(Cols))
DF= pd.DataFrame(columns=Cols)

7745


In [12]:
import datetime
print(type(str(datetime.datetime.now())))

<class 'str'>


## Metrics to Monitor

### Frobenius Norm of a model

In [13]:
def frobnorm(mod):
    S=0
    for l in mod.parameters():
        S += (l.grad**2).sum()
    S = S**0.5
    return S

### Q-quantile loss

In [14]:
def q_quantile_loss(pred, target, q):
    error = target - pred
    return torch.mean(torch.max(q * error, (q - 1) * error))

### Norm of Jacobian

In [15]:

def frobenius_norm_jacobian(model, x1,x2):
    x1 = x1.clone().detach().requires_grad_(True)
    x2 = x2.clone().detach().requires_grad_(True)
    y = model(x1,x2)
    jacobian = []
    for i in range(y[0].shape[0]):
        grad_outputs = torch.zeros_like(y[0])
        grad_outputs[:, i] = 1
        grad = torch.autograd.grad(outputs=y[0], inputs=(x1,x2), grad_outputs=grad_outputs,
                                   retain_graph=True, create_graph=True)[0]
        jacobian.append(grad)
    jacobian = torch.stack(jacobian, dim=1)
    frobenius_norm = torch.norm(jacobian, p='fro')
    return frobenius_norm.detach().item()


### Fisher Information Difference

In [16]:
import numpy as np
from scipy.stats import gaussian_kde
from scipy.integrate import quad

def kernel_density_estimate(vector):
    kde = gaussian_kde(vector)
    return kde

def finite_difference_gradient(x, kde, h=1e-5):
    pdf_value_plus_h = kde.evaluate(x + h)
    pdf_value_minus_h = kde.evaluate(x - h)
    gradient = (np.log(pdf_value_plus_h) - np.log(pdf_value_minus_h)) / (2 * h)
    return gradient

def score_function(x, kde):
    return finite_difference_gradient(x, kde)

def fisher_information_integral(kde, xmin, xmax):
    def integrand(x):
        score = score_function(x, kde)
        return score**2
    
    fisher_info, _ = quad(integrand, xmin, xmax)
    return fisher_info

def fisher_information(vector):
    kde = kernel_density_estimate(vector)
    xmin, xmax = np.min(vector), np.max(vector)
    fisher_info = fisher_information_integral(kde, xmin, xmax)
    return fisher_info

def fisher_information_difference(vec1, vec2):
    fisher_info_1 = fisher_information(vec1)
    fisher_info_2 = fisher_information(vec2)
    
    fisher_info_diff = np.abs(fisher_info_1 - fisher_info_2)
    return fisher_info_diff

## Losses To Train On

### Contractive loss

In [17]:
from torch.autograd import Variable
def loss_Contractive(W, x, recons_x, h, lam):
    dh = h * (1 - h) 

    w_sum = torch.sum(Variable(W)**2, dim=1)

    w_sum = w_sum.unsqueeze(1) # shape N_hidden x 1
 
    contractive_loss = torch.sum(torch.mm(dh**2, w_sum), 0)

    return contractive_loss.mul_(lam)

### Wasserstein Distance/Geomloss

In [18]:
from scipy.stats import wasserstein_distance
# Convert to numpy arrays
# vec1_np = vec1.numpy()
# vec2_np = vec2.numpy()

# # Compute Wasserstein distance for each pair of vectors
# wsd_list = [wasserstein_distance(vec1_np[i], vec2_np[i]) for i in range(vec1_np.shape[0])]

# # If you need an aggregate measure, you can compute the average distance
# average_wsd = sum(wsd_list) / len(wsd_list)
# average_wsd


import torch
from geomloss import SamplesLoss
vec1 = torch.rand(1,2464)
vec2 = torch.rand(1,2464)
LossWS=SamplesLoss('sinkhorn')

from scipy.stats import wasserstein_distance
# Convert to numpy arrays
vec1_np = vec1[0].numpy()
vec2_np = vec2[0].numpy()
print(LossWS(vec1,vec2), wasserstein_distance(vec1_np,vec2_np))

[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK
[pyKeOps] Compiling nvrtc binder for python ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK
tensor(204.8559) 0.005008445931719497


### Q-quantile Loss

In [19]:
class QuantileLoss(torch.nn.Module):
    def __init__(self, q):
        super(QuantileLoss, self).__init__()
        self.q = q

    def forward(self, y_pred, y_true):
        # Compute the quantile loss for each prediction
        error = y_true - y_pred
        loss = torch.maximum(self.q * error, (self.q - 1) * error)
        return loss.mean()
quantile = 0.5
q_quantile_loss = QuantileLoss(q=quantile)

### Difference in Norm of the vector

In [20]:
import torch
import torch.nn as nn

class NormDifferenceDistance(nn.Module):
    def __init__(self):
        super(NormDifferenceDistance, self).__init__()

    def forward(self, y_pred, y_true):
        # Calculate L2 norms for each tensor
        norm_pred = torch.norm(y_pred, p=2)
        norm_true = torch.norm(y_true, p=2)
        
        # Compute the absolute difference of norms as the distance
        distance = torch.abs(norm_pred - norm_true)
        return distance

class StandardizedNormDifferenceDistance(nn.Module):
    def __init__(self):
        super(StandardizedNormDifferenceDistance, self).__init__()

    def forward(self, y_pred, y_true):
        # Normalize each tensor by its L2 norm
        norm_pred = torch.norm(y_pred, p=2)
        norm_true = torch.norm(y_true, p=2)
        y_pred_normalized = y_pred / (norm_pred + 1e-8)
        y_true_normalized = y_true / (norm_true + 1e-8)
        
        # Calculate L2 distance between normalized tensors
        distance = torch.norm(y_pred_normalized - y_true_normalized, p=2)
        return distance


### Auto-regressive Loss

In [21]:
class AutoregressiveMSELoss(nn.Module):
    def __init__(self, lambda_weights=None, epsilon=1e-6):
        """
        Autoregressive MSE Loss with weighted contributions.

        Args:
            lambda_weights (list or None): Weights for the autoregressive components.
                                           If None, equal weights are used.
            epsilon (float): A small constant added to denominators for numerical stability.
        """
        super(AutoregressiveMSELoss, self).__init__()
        self.lambda_weights = lambda_weights
        self.epsilon = epsilon

    def forward(self, predictions, targets, delimiters):
        """
        Compute the autoregressive MSE loss.

        Args:
            predictions (torch.Tensor): Predicted tensor of shape [batch_size, total_length].
            targets (torch.Tensor): Target tensor of shape [batch_size, total_length].
            delimiters (list of ints): End indices for chunks, defining tensor splits.

        Returns:
            torch.Tensor: The computed loss.
        """
        n_chunks = len(delimiters)
        mse_losses = []
        start = 0

        # Compute MSE loss for each chunk
        for end in delimiters:
            pred_chunk = predictions[:, start:end]
            target_chunk = targets[:, start:end]
            mse_loss = nn.functional.mse_loss(pred_chunk, target_chunk, reduction='mean')
            mse_losses.append(mse_loss)
            start = end

        # Set default weights if not provided
        if self.lambda_weights is None:
            self.lambda_weights = [1.0] * n_chunks

        if len(self.lambda_weights) != n_chunks:
            raise ValueError("lambda_weights must have the same length as the number of chunks.")

        # Compute the weighted autoregressive loss
        autoregressive_loss = mse_losses[0]  # Start with the first chunk
        for i in range(1, n_chunks):
            autoregressive_loss += (
                self.lambda_weights[i - 1] * (mse_losses[i] / (mse_losses[i - 1] + self.epsilon))
            )
        
        return autoregressive_loss


# Example Usage
# Suppose the sequence is split into chunks based on delimiters
predictions = torch.randn(10, 2464)  # [batch_size, total_length]
targets = torch.randn(10, 2464)     # [batch_size, total_length]
delimiters = [208, 1414, 1514, 2254, 2464]

# Initialize the loss function with custom weights
loss_fn = AutoregressiveMSELoss(lambda_weights=[1,0.5, 0.7, 0.9, 1.1])

# Compute the loss
loss = loss_fn(predictions, targets, delimiters)
print("Loss:", loss.item())

Loss: 5.072986602783203


### MAPE Loss

In [22]:
class MAPE(nn.Module):
        def __init__(self):
            super(MAPE, self).__init__()
        def forward(self, vec1,vec2):
            loss = torch.mean(torch.abs((vec1 - vec2) / vec2))
            return loss

### Layer-wise-loss Normalization

In [23]:
class LWLN_loss(nn.Module):
        def __init__(self):
            super(LWLN_loss, self).__init__()
        def forward(self, vec1,vec2):
            loss = (torch.mean((vec1[:,0:208]-vec2[:,0:208])**2)/vec2[:,0:208].std() + 
                     torch.mean((vec1[:,208:1414]-vec2[:,208:1414])**2)/vec2[:,208:1414].std()+ 
                     torch.mean((vec1[:,1414:1514]-vec2[:,1414:1514])**2)/vec2[:,1414:1514].std()+
                     torch.mean((vec1[:,1514:2254]-vec2[:,1514:2254])**2)/vec2[:,1514:2254].std()+
                     torch.mean((vec1[:,2254:2464]-vec2[:,2254:2464])**2)/vec2[:,2254:2464].std())/(6)
            return loss

class LWLN_loss_single(nn.Module):
    def __init__(self):
        super(LWLN_loss_single, self).__init__()
    def forward(self, vec1,vec2):
        loss = (torch.mean((vec1[0:208]-vec2[0:208])**2)/vec2[0:208].std() + 
                 torch.mean((vec1[208:1414]-vec2[208:1414])**2)/vec2[208:1414].std()+ 
                 torch.mean((vec1[1414:1514]-vec2[1414:1514])**2)/vec2[1414:1514].std()+
                 torch.mean((vec1[1514:2254]-vec2[1514:2254])**2)/vec2[1514:2254].std()+
                 torch.mean((vec1[2254:2464]-vec2[2254:2464])**2)/vec2[2254:2464].std())/(6)
        
        return loss


### Jensen-Shannon Loss

In [24]:
import torch.nn.functional as F

class JensenShannonLoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-8):
        super(JensenShannonLoss, self).__init__()
        self.reduction = reduction
        self.eps = eps

    def forward(self, p_logits, q_logits):
        # Apply softmax to get probability distributions
        p = F.softmax(p_logits, dim=1)
        q = F.softmax(q_logits, dim=1)
        
        # Mixture distribution M = 0.5*(P + Q)
        m = 0.5 * (p + q)
        
        # Compute KL(P || M) and KL(Q || M)
        # Add eps for numerical stability in log
        kl_pm = F.kl_div(
            torch.log(m + self.eps),  # log(M)
            p,                        # P (as probabilities)
            reduction='none',
            log_target=False
        )
        kl_qm = F.kl_div(
            torch.log(m + self.eps),  # log(M)
            q,                        # Q (as probabilities)
            reduction='none',
            log_target=False
        )
        
        # Sum over classes (dim=1) to get per-sample KL divergences
        kl_pm = kl_pm.sum(dim=1)
        kl_qm = kl_qm.sum(dim=1)
        
        # Jensen-Shannon divergence: 0.5 * (KL(P||M) + KL(Q||M))
        js = 0.5 * (kl_pm + kl_qm)
        
        # Apply reduction
        if self.reduction == 'mean':
            return js.mean()
        elif self.reduction == 'sum':
            return js.sum()
        else:  # 'none'
            return js

### Frobenius Norm Jacobian (Loss)

In [25]:
class FrobeniusNormJacobian(nn.Module):
    def __init__(self, num_samples=10):
        """
        Custom loss module that calculates the Frobenius norm approximation
        of the Jacobian with respect to two inputs using Hutchinson's method.
        
        Parameters:
            num_samples (int): Number of random vectors to sample for the approximation.
        """
        super(FrobeniusNormJacobian, self).__init__()
        self.num_samples = num_samples

    def forward(self, model, inputs1, inputs2):
        """
        Forward pass to calculate the approximate Frobenius norm of the Jacobian.
        
        Parameters:
            model (torch.nn.Module): The model whose Jacobian's Frobenius norm we want to approximate.
            inputs1 (torch.Tensor): The first input tensor for which the Jacobian is computed.
            inputs2 (torch.Tensor): The second input tensor for which the Jacobian is computed.
        
        Returns:
            torch.Tensor: The approximate Frobenius norm of the Jacobian as a loss.
        """
        frobenius_norm_estimate = 0.0
        inputs1.requires_grad_(True)
        inputs2.requires_grad_(True)
        
        for _ in range(self.num_samples):
            # Generate a random vector with the same shape as model output
            random_vec = torch.randn_like(model(inputs1, inputs2)[0])
            
            # Compute the vector-Jacobian product for each input
            _, vjps1 = vjp(lambda x: model(x, inputs2)[0], inputs1, v=random_vec)
            _, vjps2 = vjp(lambda y: model(inputs1, y)[0], inputs2, v=random_vec)
            
            # Sum up the squared vjp norm for both inputs
            frobenius_norm_estimate += (vjps1.norm() ** 2 + vjps2.norm() ** 2)
        
        # Average over the number of samples and take square root
        frobenius_norm_estimate = (frobenius_norm_estimate / self.num_samples) ** 0.5
        return frobenius_norm_estimate


### Audio/Spectral Losses

#### Fourrier transform difference in Norm

In [26]:
class FFTLoss(nn.Module):
    def forward(self, x, y):
        x_fft = torch.fft.fft(x)
        y_fft = torch.fft.fft(y)
        return torch.mean(torch.abs(x_fft - y_fft))

#### Mel spectrogram Difference

In [27]:

class MelSpecL2Loss(nn.Module):
    def __init__(self, sample_rate=16000, n_fft=1024, hop_length=256, n_mels=64):
        super().__init__()
        self.mel = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=n_fft,
            hop_length=hop_length,
            n_mels=n_mels,
        )

    def forward(self, x, y):
        m1 = self.mel(x.squeeze(1))
        m2 = self.mel(y.squeeze(1))
        return F.mse_loss(m1, m2)


#### Mel Spectrogram Frechet inception distance

In [28]:
def sqrtm_newton_schulz(A, num_iter=10):
    B, N, _ = A.shape
    normA = A.norm(dim=(1, 2), keepdim=True)
    Y = A / normA
    I = torch.eye(N, device=A.device).unsqueeze(0).expand(B, -1, -1)
    Z = I.clone()

    for _ in range(num_iter):
        T = 0.5 * (3.0 * I - Z @ Y)
        Y = Y @ T
        Z = T @ Z
    return Y * torch.sqrt(normA)

class MelFIDLoss(nn.Module):
    def __init__(self, sample_rate=16000):
        super().__init__()
        self.mel = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_mels=64)

    def forward(self, x, y):
        m1 = self.mel(x.squeeze(1))  # [B, mel, T]
        m2 = self.mel(y.squeeze(1))
        f1 = m1.transpose(1, 2)  # [B, T, mel]
        f2 = m2.transpose(1, 2)
        mu1 = f1.mean(dim=1)
        mu2 = f2.mean(dim=1)
        c1 = (f1 - mu1.unsqueeze(1)).transpose(1, 2) @ (f1 - mu1.unsqueeze(1)) / (f1.shape[1] - 1)
        c2 = (f2 - mu2.unsqueeze(1)).transpose(1, 2) @ (f2 - mu2.unsqueeze(1)) / (f2.shape[1] - 1)
        sqrt_cov = sqrtm_newton_schulz(c1 @ c2)
        trace_term = torch.diagonal(c1 + c2 - 2 * sqrt_cov, dim1=1, dim2=2).sum(dim=1)
        mean_term = (mu1 - mu2).pow(2).sum(dim=1)
        return (mean_term + trace_term).mean()


## List of Losses to train on

In [29]:
import itertools

# Initial list of losses
Losses=["sinkhorn","MAPE","MAE","MSE","AUTO",
            "KL divergence",
            "Mel L2","Mel FID","FFT Loss",
               "LWLN","LWWS","Latent","Forb_norm","CAE","JS Loss", "ws_scipy",
               "gw_loss","LWWS_scipy",
              "Q-quantile_loss","FIM","log-norm"]
all_losses=Losses
print(all_losses)


['sinkhorn', 'MAPE', 'MAE', 'MSE', 'AUTO', 'KL divergence', 'Mel L2', 'Mel FID', 'FFT Loss', 'LWLN', 'LWWS', 'Latent', 'Forb_norm', 'CAE', 'JS Loss', 'ws_scipy', 'gw_loss', 'LWWS_scipy', 'Q-quantile_loss', 'FIM', 'log-norm']


## Method to update all metrics+losses onto wandb after a forward pass

In [30]:
Losses=["MAPE","MAE","MSE","Latent","sinkhorn",
        "gw_loss","Mel L2","Mel FID","FFT Loss",
        "ws_scipy","CAE","Q-quantile_loss","LWLN","LWWS",
        "FIM","log-norm","AUTO","KL divergence","Forb_norm","JS Loss",
        "LWWS_scipy","ws_scipy"]
        #py_riemanian , geodesic
from torch.nn.functional import pairwise_distance
#from torch.profiler import profile, record_function, ProfilerActivity


vec1 = torch.rand(1,2464).cuda()
vec2 = torch.rand(1,2464).cuda()

def update_all_metrics(d_loos,mod,vec1,vec2,tg,DF):
    Losses=["Mel L2","Mel FID","FFT Loss","JS Loss","MAE","MSE","Latent","MAPE","sinkhorn","gw_loss","ws_scipy","CAE","Q-quantile_loss","LWLN","LWWS","FIM","log-norm","AUTO","KL divergence","Forb_norm","LWWS_scipy","ws_scipy 0.9","ws_scipy"]
    Lambda=0.05
    #with torch.autocast("cuda"):
    output=mod(vec1,vec2)
    for Loss in Losses:
        if Loss=="FFT Loss":
            FFT=FFTLoss()
            loss_tr=FFT(output[0].cpu(),tg.cpu())
            wandb.log({Loss:loss_tr})

            wandb.log({"FFT 1-t":FFT(vec1.cpu(),tg.cpu())})
            wandb.log({"FFT 2-t":FFT(vec2.cpu(),tg.cpu())})
            wandb.log({"FFT 1-p":Mel2(output[0].cpu(),vec1.cpu())})
            wandb.log({"FFT 2-p":Mel2(output[0].cpu(),vec2.cpu())})
            
            d_loos[Loss]=loss_tr

        if Loss=="Mel FID":
            FID=MelFIDLoss()
            loss_tr=FID(output[0].cpu(),tg.cpu())

            wandb.log({"FID 1-t":FID(vec1.cpu(),tg.cpu())})
            wandb.log({"FID 2-t":FID(vec2.cpu(),tg.cpu())})
            wandb.log({"FID 1-p":Mel2(output[0].cpu(),vec1.cpu())})
            wandb.log({"FID 2-p":Mel2(output[0].cpu(),vec2.cpu())})
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr

        if Loss== "Mel L2":
            Mel2=MelSpecL2Loss()

            wandb.log({"Mel2 1-t":Mel2(vec1.cpu(),tg.cpu())})
            wandb.log({"Mel2 2-t":Mel2(vec2.cpu(),tg.cpu())})
            wandb.log({"Mel2 1-p":Mel2(output[0].cpu(),vec1.cpu())})
            wandb.log({"Mel2 2-p":Mel2(output[0].cpu(),vec2.cpu())})
            
            loss_tr=Mel2(output[0].cpu(),tg.cpu())
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss== "JS Loss":
            js_loss = JensenShannonLoss()
            loss_tr=js_loss(output[0].cpu(),tg.cpu())
            wandb.log({Loss:loss_tr})
        if Loss=="Latent":
            mse_loss = torch.nn.MSELoss()
            z1_vec1=mod.enc1(vec1)[0]#[0,:,:]
            z1_vec2=mod.enc1(vec2)[0]#[0,:,:]
            z1_pred=mod.enc1(output[0])[0]#[0,:,:]
            z1_tg=mod.enc1(tg)[0]
            
            out3=torch.cat([z1_tg,z1_tg], dim=2)
            Z1_tg = mod.tanh(mod.vec2neck(torch.sum(out3, dim=1, keepdim=False))).to("cpu")
            
            l2_z11 = mse_loss(z1_vec1, z1_tg)
            l2_z12 = mse_loss(z1_vec2,z1_tg)
            l2_z1p = mse_loss(z1_pred,z1_tg)
            del(z1_vec1)
            del(z1_vec2)
            del(z1_pred)
            del(z1_tg)
            
            z2_vec1=mod.enc2(vec1)[0]#[0,:,:]
            z2_vec2=mod.enc2(vec2)[0]#[0,:,:]
            z2_pred=mod.enc2(output[0])[0]#[0,:,:]
            z2_tg=mod.enc2(tg)[0]#[0,:,:]
            
            
            out3=torch.cat([z2_tg,z2_tg], dim=2)
            Z2_tg = mod.tanh(mod.vec2neck(torch.sum(out3, dim=1, keepdim=False))).to("cpu")
            
            l2_z21 = mse_loss(z2_vec1, z2_tg)
            l2_z22 = mse_loss(z2_vec2,z2_tg)
            l2_z2p = mse_loss(z2_pred,z2_tg)
            del(z2_vec1)
            del(z2_vec2)
            del(z2_pred)
            del(z2_tg)
            
            LZ1= mse_loss(Z1_tg.cuda() ,output[1].cuda())
            LZ2=mse_loss(Z2_tg.cuda(),output[1].cuda())
            
            LZ = LZ1 +LZ2
            del(Z1_tg)
            del(Z2_tg)
            loss_tr =LZ +(2.0*l2_z1p/(l2_z11 + l2_z12))+(2.0*l2_z2p/(l2_z21 + l2_z22))

            wandb.log({"Branch 1  x1-p MSE Z":l2_z11})
            wandb.log({"Branch 1  x2-p MSE Z":l2_z12})
            wandb.log({"Branch 1  p-t MSE Z":l2_z1p})
            
            wandb.log({"Branch 2  x1-p MSE Z":l2_z21})
            wandb.log({"Branch 2  x2-p MSE Z":l2_z22})
            wandb.log({"Branch 2  p-t MSE Z":l2_z2p})

            wandb.log({"Branch 1  before after merge":LZ1})
            wandb.log({"Branch 2  before after merge":LZ2})


            
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
            mod=mod.cuda()
        if Loss=="MAPE":
            mape=MAPE()
            loss_tr = mape(output[0],tg)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="MAE":
            mae = nn.L1Loss()
            loss_tr = mae(output[0],tg)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="MSE":
            loss_tr = criterion(output[0],tg)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="sinkhorn":
            loss_tr=LossWS(output[0],tg)
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="AUTO":
            delimiters = [208, 1414, 1514, 2254, 2464]
            predictions=output[0]
            targets=tg
            # Initialize the loss function with custom weights
            loss_fn = AutoregressiveMSELoss(lambda_weights=[10,5,2, 1, 0.5])

            # Compute the loss
            loss_tr = loss_fn(predictions, targets, delimiters)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="ws_scipy 0.9":
            vec1_np = output[0].detach().cpu().numpy()
            vec2_np = tg.detach().cpu().numpy()
            wsd_list = [wasserstein_distance(vec1_np[i], vec2_np[i]) for i in range(vec1_np.shape[0])]
            average_wsd = sum(wsd_list) / len(wsd_list)
            loss_tr = average_wsd *0.9
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="ws_scipy":
            vec1_np = output[0].detach().cpu().numpy()
            vec2_np = tg.detach().cpu().numpy()
            wsd_list = [wasserstein_distance(vec1_np[i], vec2_np[i]) for i in range(vec1_np.shape[0])]
            average_wsd = sum(wsd_list) / len(wsd_list)
            loss_tr = average_wsd *0.9
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="gw_loss":
            try:
                if len(DF)>0:
                    PD=DF[Cols[2466:4930]]
                    dataPD=PD.to_numpy()
                    GT=DF[Cols[2:2466]]
                    dataGT=GT.to_numpy()
                    FN=DF[Cols[4930:7394]]
                    dataF=FN.to_numpy()
                    # Define filter function – can be any scikit-learn transformer
                    filter_func = Projection(columns=[0,1])
                    # Define cover
                    cover = CubicalCover(n_intervals=20, overlap_frac=0.5)
                    # Choose clustering algorithm – default is DBSCAN
                    clusterer = DBSCAN()
    
                    # Configure parallelism of clustering step
                    n_jobs = 8
    
                    # Initialise pipeline
                    pipe = make_mapper_pipeline(
                        filter_func=filter_func,
                        cover=cover,
                        clusterer=clusterer,
                        verbose=True,
                       n_jobs=n_jobs)
                    graphT=pipe.fit_transform(dataGT)
                    graphP = pipe.fit_transform(dataPD)
                    # Transform the second point cloud using the fitted pipeline
    
                    T=graphT.to_networkx()
                    P=graphP.to_networkx()
    
                    # Convert graphs to adjacency matrices
                    def graph_to_adjacency_matrix(graph):
                        return torch.tensor(nx.to_numpy_array(graph), dtype=torch.float32)
    
                    # Assume G1 and G2 are your two NetworkX graph objects
                    adj1 = graph_to_adjacency_matrix(T)
                    adj2 = graph_to_adjacency_matrix(P)
    
                    def gw_loss(C1, C2, P):
                        C1P = torch.matmul(C1, P)  # n x m matrix
                        PC2 = torch.matmul(P, C2)  # n x m matrix
                        return torch.sum((C1P - PC2) ** 2)
                    # Compute pairwise distance matrices for each graph
                    C1 = torch.cdist(adj1, adj1, p=2)  # Pairwise distance matrix for G1
                    C2 = torch.cdist(adj2, adj2, p=2)  # Pairwise distance matrix for G2
                    # Initialize coupling matrix with uniform distribution
                    n, m = C1.shape[0], C2.shape[0]
                    P = torch.ones((n, m), dtype=torch.float32, requires_grad=True) / (n * m)
                    P=P.detach()
                    P.requires_grad=True
    
                    P=P.detach()
                    P.requires_grad=True
                    #print(P.requires_grad)
                    ##optimizer = torch.optim.Adam([P], lr=0.01)
                    ##optimizer.step()
                    loss_tr=gw_loss(C1,C2,P)*0.0005
                else:
                    loss_tr=0
            except Exception as e:
                continue
            
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="CAE":
            W=mod.vec2neck.weight
            CL=loss_Contractive(W,output[0],tg, output[1], Lambda)
            loss_tr = CL
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="Q-quantile_loss":    
            loss_tr=q_quantile_loss(output[0],tg).cuda()
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="LWLN":
            LW=LWLN_loss()
            loss_tr=LW(output[0],tg)
            #print(Loss,": \t",loss_tr
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="LWWS":
            loss_tr=LossWS(output[0][:,0:208],tg[:,0:208]) +LossWS(output[0][:,208:1414],tg[:,208:1414])+LossWS(output[0][:,1414:1514],tg[:,1414:1514])+LossWS(output[0][:,1514:2254],tg[:,1514:2254])+LossWS(output[0][:,2254:2464],tg[:,2254:2464])
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
            
            
        if Loss=="LWWS_scipy":
            vec1_np = output[0].detach().cpu().numpy()
            vec2_np = tg.detach().cpu().numpy()
            
            delimiters = [208, 1414, 1514, 2254, 2464]
            # Compute Wasserstein distance for each chunk and sum the averages
            total_average_wsd = 0
            start = 0
            for end in delimiters:
                chunk1 = vec1_np[:, start:end]
                chunk2 = vec2_np[:, start:end]
                wsd_list = [wasserstein_distance(chunk1[i], chunk2[i]) for i in range(chunk1.shape[0])]
                average_wsd = sum(wsd_list) / len(wsd_list)  # Average Wasserstein distance for the current chunk
                total_average_wsd += average_wsd  # Add to total
                start = end
            # Print the total sum of batch averages
            #print(f"Total Sum of Batch Averages across all chunks: {total_average_wsd}")

            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:total_average_wsd})
            d_loos[Loss]=total_average_wsd
        if Loss=="FIM":
            # Example numpy arrays
            output[0].detach().requires_grad=True
            tg.requires_grad=True
            # Step 1: Convert to probabilities and compute log-probabilities with gradient tracking
            log_probs1 = torch.log_softmax(output[0], dim=1)
            log_probs2 = torch.log_softmax(tg, dim=1)

            # Step 2: Convert log-probs to probs while ensuring gradients are tracked
            probs1 = log_probs1.exp()
            probs2 = log_probs2.exp()

            # Step 3: Compute gradients of log-probabilities with respect to original tensors
            # Using create_graph=True to allow second-order gradients
            grad_log_probs1 = torch.autograd.grad(outputs=log_probs1, inputs=output[0], grad_outputs=torch.ones_like(log_probs1), retain_graph=True, create_graph=True)[0]
            grad_log_probs2 = torch.autograd.grad(outputs=log_probs2, inputs=tg, grad_outputs=torch.ones_like(log_probs2), retain_graph=True, create_graph=True)[0]

            # Step 4: Approximate Fisher Information for each tensor
            fim1 = probs1 * grad_log_probs1**2
            fim2 = probs2 * grad_log_probs2**2

            # Step 5: Compute Fisher Distance (use Frobenius norm)
            fisher_distance = torch.norm(fim1 - fim2, p='fro')  # Frobenius norm for matrix difference
            loss_tr=fisher_distance*100
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="log-norm":
            N1=NormDifferenceDistance()
            loss_tr=N1(output[0],tg)
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="KL divergence":
            kl_loss = nn.KLDivLoss(reduction="batchmean",log_target=True)
            loss_tr=kl_loss(output[0],nn.functional.log_softmax(tg))*0.1
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
        if Loss=="Forb_norm":
            f=FrobeniusNormJacobian(10)
            loss_tr=f(mod,output[0],tg)*0.015
            #print(Loss,": \t",loss_tr)
            wandb.log({Loss:loss_tr})
            d_loos[Loss]=loss_tr
    return d_loos,output,mod

# Creating a shared meta-learner starting checkpoint

In [31]:
# BrainTransformer=TransformerAE(max_seq_len=50,
#             N=4,
#             heads=4,
#             d_model=960,#960
#             d_ff=960,#960
#             neck=512,#256
#             dropout=0.07#0.12
#             )

# torch.save({'epoch':-1,'model_state_dict': BrainTransformer.state_dict()},
#             './AE epoch{} {} {}.pth'.format( "initial" , "original", -1))
# del(BrainTransformer)

In [32]:
string='overlapping 0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 11'


# Training Loop

In [33]:


           
track=0

from torchinfo import summary

warnings.filterwarnings("ignore")


global track , output , Brain ,model
grads=None


accelerator = Accelerator(mixed_precision="bf16")
nb_scen=len(os.listdir("./data/Scenario/"))
for t in range(nb_scen):
    namef=os.listdir("./data/Scenario/")[t]
    train_pair2 = np.load("./data/Scenario/"+os.listdir("./data/Scenario/")[t]+"/train pair.npy", allow_pickle=True)
    print("./data/Scenario/"+os.listdir("./data/Scenario/")[t]+"/"+"train pair.npy" ,len(train_pair2))
    val_pair2 = np.load("./data/Scenario/"+os.listdir("./data/Scenario/")[t]+"/val pair.npy", allow_pickle=True)
    test_pair2 = np.load("./data/Scenario/"+os.listdir("./data/Scenario/")[t]+"/test pair.npy", allow_pickle=True)
    
    train_pair2 = [ list(x) for x in train_pair2]
    test_pair2 = [ list(x) for x in test_pair2]
    val_pair2 = [ list(x) for x in val_pair2]
    
    #random.shuffle(test_pair2)
    #random.shuffle(val_pair2)
    
    #if len(train_pair2)>1000 and  len(val_pair2)>1000 and  len(test_pair2)>100:
    if namef=='overlapping 0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 11 gelu' :
        for combination in all_losses:
            d_loos=dict()
            DF= pd.DataFrame(columns=Cols)
            track=0


            cnn_samples=20
            num_epochs=800
            batch_size=36
            batch_limit=10000
            epochs_fn=5

            if not(os.path.isdir(f".//Experiments//{os.listdir('.//data//Scenario//')[t]}")):
                os.mkdir(f".//Experiments//{os.listdir('.//data//Scenario//')[t]}")
            results_path=f".//Experiments//{os.listdir('.//data//Scenario//')[t]}//{combination } {str(datetime.datetime.now())} {num_epochs} /"
            if not(os.path.isdir(results_path)):
                os.mkdir(results_path)
            if not(os.path.isdir(results_path+f'Tracking/')):
                os.mkdir(results_path+f'Tracking/')
            if not(os.path.isdir(results_path+f'Attention/')):
                os.mkdir(results_path+f'Attention/')
    
            cnn_acc_ID=[]
            cnn_acc_OOD=[]
            
            Lambda=0
            step_size=0
            factor=0
            threshhold=0
            threshold_mode = 0
            eps=0
            try:
                del(mod)
                del(optimizerEnc1)
                del(optimizerEnc2) 
                del(optimizerDense)
                del(optimizerDec)
            except Exception as e:
                print(f"--------------------{str(datetime.datetime.now())} {e}")
            ch=torch.load('./AE epoch{} {} {}.pth'.format( "initial" , "original", -1),map_location="cpu")
            mod=TransformerAE(max_seq_len=50,
                                    N=4,
                                    heads=4,
                                    d_model=960,#960
                                    d_ff=960,#960
                                    neck=512,#256
                                    dropout=0.07)#0.12)
            mod.load_state_dict(ch['model_state_dict'])
            del(ch)
            minimal_loss=100000
            print(summary(mod))
            lrE1=0.15 #trial.suggest_float("Learning_rate",0.0002,0.5)
            lrE2=0.04 
            lrL=0.15 
            lrD=0.085
            optimizerEnc1 = Adam(mod.enc1.parameters(), lr=lrE1,eps=1e-10,weight_decay=0.005)
            optimizerEnc2 = Adadelta(mod.enc2.parameters(), lr=lrE2,eps=1e-10,weight_decay=0.005)
            optimizerDense = SGD(mod.vec2neck.parameters(), lr=lrL,weight_decay=0.005)
            optimizerDec = Adadelta(mod.dec.parameters(), lr=lrD,eps=1e-10,weight_decay=0.01)
            mod = torch.compile(mod)
    
    
    
            sched_name="CyclicLR"
            if sched_name=="CyclicLR" :
                step_size=24000#trial.suggest_int("step_size_up",900,2800)
                schedulerEnc1 = torch.optim.lr_scheduler.CyclicLR(optimizerEnc1, base_lr=1e-4, max_lr=lrE1, step_size_up=step_size, step_size_down=80000,scale_mode="iterations",mode="triangular2",cycle_momentum=False)
                schedulerEnc2 = torch.optim.lr_scheduler.CyclicLR(optimizerEnc2, base_lr=1e-4, max_lr=lrE2, step_size_up=step_size, step_size_down=80000,scale_mode="iterations",mode="triangular2",cycle_momentum=False)
                scheduler = torch.optim.lr_scheduler.CyclicLR(optimizerDense, base_lr=1e-4, max_lr=lrD, step_size_up=8000, step_size_down=8000,scale_mode="iterations",mode="triangular",cycle_momentum=False)

    
            resume_epoch=0
            # ch=torch.load("/media/crns/ADATA HD330/Experiments/mixed model/AE epoch 0 600.pth",map_location="cpu")
            # resume_epoch=ch["epoch"]
            # mod.load_state_dict(ch['model_state_dict'])
            # optimizerEnc1.load_state_dict(ch['optimizerENC1_state_dict'])
            # optimizerEnc2.load_state_dict(ch['optimizerENC2_state_dict'])
            # optimizerDense.load_state_dict(ch['optimizerDense_state_dict'])
            # optimizerDec.load_state_dict(ch['optimizerDec_state_dict'])
    
            #del(ch)
    
            criterion = nn.MSELoss()
            LW=LWLN_loss()
            LossWS=SamplesLoss('sinkhorn')
            LWs=LWLN_loss_single()
    
            device = accelerator.device
            run = wandb.init(
            # Set the project where this run will be logged
            project="WR4CL-project",
            name= f"{str(datetime.datetime.now())} {combination} {namef[12:16]}-{namef[42:44]} {num_epochs}" ,
            # Track hyperparameters and run metadata
            config={
                "Scenario":namef,
                "instances in train":len(train_pair2),
                "Loss":combination,
                "lr Encoder 1": lrE1,
                "lr Encoder 2": lrE2,
                "lr Linear": lrL,
                "lr Decoder": lrD,
                "epochs": num_epochs,
                "sched_name": sched_name,
                "step_size": step_size,
                "batch_size": batch_size,
                "step_size":step_size,
                "Lambda_contractive":Lambda
            },)
    
            #run = wandb.init(project="aymen-project", id="ar1497fk", resume="must")
    
            # "factor":factor,
            #     "threshhold":threshhold,
            #     "threshold_mode":threshold_mode,
            #     "eps":eps,
            #random.shuffle(train_pair2)
            cs_tr=CustomDataset(train_pair2,batch_size=batch_size,batch_limit=batch_limit)
            cs_tr=accelerator.prepare(cs_tr)
            
            nb_batches = len(cs_tr)
            
            print("Number of training batchs:",nb_batches)
            cs_val=CustomDataset(val_pair2,batch_size=batch_size,batch_limit=batch_limit)
            nb_val_batches = len(cs_val)

            cs_ts=CustomDataset(test_pair2,batch_size=batch_size,batch_limit=batch_limit)
    
            mod, optimizerEnc1,optimizerEnc2,optimizerDense,optimizerDec, cs_tr= accelerator.prepare(mod, optimizerEnc1,optimizerEnc2,optimizerDense,optimizerDec,cs_tr)
            # wandb.watch(mod, log_freq=10000 ,criterion=criterion,
            #     log='parameters',
            #     log_graph=True)
            #mod.train()
            
            mod.train()
            
            wandb.define_metric("custom_step")
            wandb.define_metric("Prediction Histogram", step_metric='custom_step')
            wandb.define_metric("Total Histogram", step_metric='custom_step')
            used_Loss=0.0
            for epoch in tqdm(range(resume_epoch,num_epochs),total=num_epochs):
                # for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(locals().items())), key= lambda x: -x[1])[:10]:
                #     print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
                wandb.log({f"Current epoch":epoch})
                random.shuffle(train_pair2)
                
                #start_time_epoch = time.time()
                for i in range(nb_batches):#nb_batches for training'''
                    start_time = time.time()
                    
                    Dataset,EXP,ACC,U = cs_tr[i]
                    
                    x1,x2,tg = Dataset[:,0,:], Dataset[:,1,:],Dataset[:,2,:]
    
                    x1=x1.cuda() #.to(torch.float32)
                    x2=x2.cuda() #.to(torch.float32)
                    tg=tg.cuda() #.to(torch.float32)
    
                    optimizerEnc1.zero_grad()
                    optimizerEnc2.zero_grad()
                    optimizerDense.zero_grad()
                    optimizerDec.zero_grad()
    
                    #output = mod(x1,x2)
    
                    d_loos,output,mod=update_all_metrics(d_loos,mod,x1,x2,tg,DF)
                    torch.nn.utils.clip_grad_norm_(mod.parameters(), max_norm=2.0)
    
    
                    used_Loss=0
                    #print(used_Loss,d_loos)
                    if isinstance(combination, list):
                        for pair in d_loos.items():
                            if pair[0]==combination[0] or pair[0]==combination[1] :
                                used_Loss=used_Loss+pair[1]
                    if isinstance(combination, str):
                        for pair in d_loos.items():
                            if pair[0]==combination: 
                                used_Loss=used_Loss+pair[1]
                    #print(combination ,"\t \t",used_Loss)
    
                    accelerator.backward(used_Loss)
                    #loss_tr.backward()
                    frob = frobnorm(mod)
                    optimizerEnc1.step()
                    optimizerEnc2.step()
                    optimizerDense.step()
                    optimizerDec.step()
                    if sched_name=="ReduceLROnPlateau" :
                        scheduler.step(used_Loss)
                    else:
                        scheduler.step()
                        schedulerEnc1.step()
                        schedulerEnc2.step()
                    
                    loss_to_save = float(used_Loss.detach().cpu().item())
                    wandb.log({f"Tracked Loss {combination}":loss_to_save})
                
                
                
                if used_Loss.detach().cpu().item()<minimal_loss:
                    minimal_loss=used_Loss.detach().cpu().item()
                    torch.save({'epoch':epoch,'model_state_dict': mod.state_dict(),
                                'optimizerENC1_state_dict':  optimizerEnc1.state_dict() ,
                                'optimizerENC2_state_dict':optimizerEnc2.state_dict(),
                                'optimizerDense_state_dict':optimizerDense.state_dict(),
                                'optimizerDec_state_dict': optimizerDec.state_dict(),
                                'Batch Loss':used_Loss.detach().cpu().item()},
                                results_path+'AE epoch best_.pth')
    
                end_time = time.time() 
                execution_time = end_time - start_time  
                #if execution_time>4000 :
                #    break
                if (epoch+1)%50==0 or (epoch+1) in [1,5,10,30]:
                    print("validating")
                    for block in [2,3,4]:
                            for head in range(4):
                                plt.figure(figsize=(20, 20))
                                hm=sns.heatmap(torch.mean( torch.mean(output[block][head], dim=1), dim=0).detach().cpu(), annot=False, cmap='cubehelix')
                                plt.title('Attention Heatmap')
                                heatmap_path = f'heatmap {block-2}_{head}_epoch{epoch}_step_{i}.png'
                                plt.savefig(results_path+"Attention/"+heatmap_path,format='svg', dpi=800)
                                wandb.log({f"attention_heatmap {block-2}_{head}":  wandb.Image(hm,caption=f"attention_heatmap attention_heatmap {block-2}_{head}")})
                                plt.close()
                    mod.eval()
                    loss_val = []
                    for i_val in range(nb_val_batches):#nb_val_batches batches for validation'''
                        #start_time_batch = time.time()
                        Dataset_val,EXP_val,ACC_val,U_val = cs_val[i_val]
                        x1_val,x2_val,tg_val = Dataset_val[:,0,:], Dataset_val[:,1,:],Dataset_val[:,2,:]
                        x1_val=x1_val.cuda() #.to(torch.float32)
                        x2_val=x2_val.cuda() #.to(torch.float32)
                        tg_val=tg_val.cuda() #.to(torch.float32)
                        with torch.no_grad():
                            #output_val = mod(x1_val,x2_val)
                            
                            loss_val.append(criterion(mod(x1_val,x2_val)[0],tg_val))
                            #print(x1_val.shape,mod(x1_val,x2_val)[0].shape)
                    loss_value = sum(loss_val)/len(loss_val)
                    wandb.log({f"val_loss":loss_value})
                    print("entering test set")
                    DF= pd.DataFrame(columns=Cols)
                    track=0
                    nb_test_batches = len(cs_ts)
                    
                    mod,cs_ts= accelerator.prepare(mod, cs_ts)
                    mod.eval()
    
                    #loss_values = []
                    
                    nb_test= min(60,nb_test_batches)
                    print("test batches :",nb_test_batches,"batch size",batch_size,"temporarly testing on batchs:",nb_test)
                    for j in range(nb_test): #nb_test_batches number of batches for testing'''
                        Dataset,EXP,ACC,U = cs_ts[j]
                        x1,x2,tg = Dataset[:,0,:], Dataset[:,1,:],Dataset[:,2,:]
    
                        x1=x1.cuda() #.to(torch.float32)
                        x2=x2.cuda() #.to(torch.float32)
                        tg=tg.cuda() #.to(torch.float32)
                        #with torch.no_grad():
                        #    output = mod(x1,x2)
                        #    loss_values.append(criterion(output[0],tg))
    
                        for vec in range(len(x1)):
                            DF.at[track,"label task 1"]=f'{EXP[vec][0]}'
                            DF.at[track,"label task 2"]=f'{EXP[vec][1]}'
                            #print(Cols[2:2466][-2:],Cols[2466:4930][-2:],Cols[4930:7394][-2:])
    
                            vec1=mod(x1,x2)[0][vec].detach().cpu() #prediction
                            vec2=tg[vec].detach().cpu()            #target
                            DF.loc[track,Cols[2:2466]]=vec2.tolist()
                            DF.loc[track,Cols[2466:4930]]=vec1.tolist()#Cols[4930:7394][-2:])
    
    
    
                            MSE=criterion(vec1,vec2).item()
                            MS1=criterion(vec1[:208],vec2[:208]).item()
                            MS2=criterion(vec1[208:1414],vec2[208:1414]).item()
                            MS3=criterion(vec1[1414:1514],vec2[1414:1514]).item()
                            MS4=criterion(vec1[1514:2254],vec2[1514:2254]).item()
                            MS5=criterion(vec1[2254:],vec2[2254:]).item()
                            #print("MSE :",MS1,MS2,MS3,MS4,MS5,MSE)
                            DF.at[track,"MSE"]=MSE
                            DF.at[track,"MSE 1"]=MS1
                            DF.at[track,"MSE 2"]=MS2
                            DF.at[track,"MSE 3"]=MS3
                            DF.at[track,"MSE 4"]=MS4
                            DF.at[track,"MSE 5"]=MS5
    
    
    
                            kl_loss = nn.KLDivLoss(reduction="mean")
                            KLD=kl_loss(vec1,vec2).item() #pred, true
                            KL1=kl_loss(vec1[:208],vec2[:208]).item()
                            KL2=kl_loss(vec1[208:1414],vec2[208:1414]).item()
                            KL3=kl_loss(vec1[1414:1514],vec2[1414:1514]).item()
                            KL4=kl_loss(vec1[1514:2254],vec2[1514:2254]).item()
                            KL5=kl_loss(vec1[2254:],vec2[2254:]).item()
                            #print("KLD :",KLD,KL1,KL2,KL3,KL4,KL5)
                            DF.at[track,"KLD"]=KLD
                            DF.at[track,"KL 1"]=KL1
                            DF.at[track,"KL 2"]=KL2
                            DF.at[track,"KL 3"]=KL3
                            DF.at[track,"KL 4"]=KL4
                            DF.at[track,"KL 5"]=KL5
    
    
    
    
                            WSD=wasserstein_distance(vec1,vec2)
                            WS1=wasserstein_distance(vec1[:208],vec2[:208])
                            WS2=wasserstein_distance(vec1[208:1414],vec2[208:1414])
                            WS3=wasserstein_distance(vec1[1414:1514],vec2[1414:1514])
                            WS4=wasserstein_distance(vec1[1514:2254],vec2[1514:2254])
                            WS5=wasserstein_distance(vec1[2254:],vec2[2254:])
                            #print("WSD :",WSD,WS1,WS2,WS3,WS4,WS5)
                            DF.at[track,"Wasserstein Loss"]=WSD
                            DF.at[track,"WS 1"]=WS1
                            DF.at[track,"WS 2"]=WS2
                            DF.at[track,"WS 3"]=WS3
                            DF.at[track,"WS 4"]=WS4
                            DF.at[track,"WS 5"]=WS5
    
                            for name, param in mod.named_parameters():
                                if name == 'vec2neck.weight':
                                    W = param
                                    break
    
                            CL=loss_Contractive(mod.vec2neck.weight ,vec1,vec2, output[1], 0.00001)
                            DF.at[track,"contractive distance"]=CL.cpu().item()
    
                            #print("Contractive :",CL.cpu().item())
                            Norm1=np.sum(np.abs(vec1.numpy()))
                            N11=np.sum(np.abs(vec1[:208].numpy()))
                            N12=np.sum(np.abs(vec1[208:1414].numpy()))
                            N13=np.sum(np.abs(vec1[1414:1514].numpy()))
                            N14=np.sum(np.abs(vec1[1514:2254].numpy()))
                            N15=np.sum(np.abs(vec1[2254:].numpy()))
                            #print("Weight pred L1: ",Norm1, N11,N12,N13,N14,N15)
                            DF.at[track,"N1"]=Norm1
                            DF.at[track,"N11"]=N11
                            DF.at[track,"N12"]=N12
                            DF.at[track,"N13"]=N13
                            DF.at[track,"N14"]=N14
                            DF.at[track,"N15"]=N15
    
                            Norm2=np.sum(np.abs(vec2.numpy()))
                            N21=np.sum(np.abs(vec2[:208].numpy()))
                            N22=np.sum(np.abs(vec2[208:1414].numpy()))
                            N23=np.sum(np.abs(vec2[1414:1514].numpy()))
                            N24=np.sum(np.abs(vec2[1514:2254].numpy()))
                            N25=np.sum(np.abs(vec2[2254:].numpy()))
                            #print("Weight GT L1: ",Norm2, N21,N22,N23,N24,N25)
                            DF.at[track,"N2"]=Norm2
                            DF.at[track,"N21"]=N21
                            DF.at[track,"N22"]=N22
                            DF.at[track,"N23"]=N23
                            DF.at[track,"N24"]=N24
                            DF.at[track,"N25"]=N25
    
                            DF.at[track,"saturated in pred(%)"]=100*sum(1 for x in vec1 if x > 0.95 or x<0.05)/len(vec1)
                            DF.at[track,"saturated in GT(%)"]=100*sum(1 for x in vec2 if x > 0.95 or x<0.05)/len(vec2)
                            DF.at[track,"LWLN"]=LWs(vec1,vec2).cpu().item()
                            if vec<cnn_samples and j==0:
                                print(f"Test vector {vec} batch {j} Epoch: {epoch}")
                                figure=get_plot_object(vec1, tg)
                                wandb.log({"Plot of pred vs target":  wandb.Image(figure,caption=f"Test vector {vec} batch {j} Epoch: {epoch} vs pred")})
                                plt.close(figure)
    
                                #import umap
                                # reducer = umap.UMAP()
                                # Batch_UMAP=torch.rand(80,2464)
                                # fitted_reduced = reducer.fit(Batch_UMAP)
                                # fitted_reduced.transform(vec1)
        
                                ###########RECONSTRUCTING##############
                                print("Entering Reconstruction for fine-tuning and eigenvalues")
                                y_pred=torch.unsqueeze(output[0][vec], 0) 
                                y =torch.unsqueeze(tg[vec], 0) 
        
                                selected_row = cs_ts.df.iloc[int(U[vec][0]), 11:17]  
                                columns_with_one = selected_row[selected_row == 1].index.tolist()
                                activ=columns_with_one
                                epochCNN=cs_ts.df.loc[int(U[vec][0])]['epoch']
        
        
                                checkpoint=OrderedDict()
                                checkpoint1=OrderedDict()
                                checkpoint2=OrderedDict()
                                vector_aux= output[0][vec].detach()
                                y_pred=vector_aux.cpu()
        
                                task1=[int(x) for x in EXP[vec][0]]
                                task2=[int(x) for x in EXP[vec][1]]
                                task3=sorted(list(set(task1+task2)))
        
        
                                All=list(range(10))
                                L2=[k for k in All if k not in task3] # Out of distribution classes
                                L_others=[k for k in All if k not in task3] #Classes to test on (In distribution)
        
                                checkpoint["module_list.0.weight"]=torch.tensor(np.array(y_pred[0:200]).reshape([8, 1, 5, 5]))
                                checkpoint["module_list.0.bias"]=torch.tensor(np.array(y_pred[200:208]).reshape([8]))
        
                                checkpoint["module_list.3.weight"]=torch.tensor(np.array(y_pred[208:1408]).reshape([6, 8, 5, 5]))
                                checkpoint["module_list.3.bias"]=torch.tensor(np.array(y_pred[1408:1414]).reshape([6]))
        
                                checkpoint["module_list.6.weight"]=torch.tensor(np.array(y_pred[1414:1510]).reshape([4, 6, 2, 2]))
                                checkpoint["module_list.6.bias"]=torch.tensor(np.array(y_pred[1510:1514]).reshape([4]))
        
                                checkpoint["module_list.9.weight"]=torch.tensor(np.array(y_pred[1514:2234]).reshape([20,36]))
                                checkpoint["module_list.9.bias"]=torch.tensor(np.array(y_pred[2234:2254]).reshape([20]))
        
                                checkpoint["module_list.11.weight"]=torch.tensor(np.array(y_pred[2254:2454]).reshape([10,20]))
                                checkpoint["module_list.11.bias"]=torch.tensor(np.array(y_pred[2454:2464]).reshape([10]))
        
                                Brain = CNN(1,activ[0],0,"kaiming_uniform")
        
                                model=copy.deepcopy(Brain)
                                model.load_state_dict(checkpoint)
                           
                           
                                
                                checkpoint1["module_list.0.weight"]=torch.tensor(np.array(x1[vec][0:200].cpu()).reshape([8, 1, 5, 5]))
                                checkpoint1["module_list.0.bias"]=torch.tensor(np.array(x1[vec][200:208].cpu()).reshape([8]))
        
                                checkpoint1["module_list.3.weight"]=torch.tensor(np.array(x1[vec][208:1408].cpu()).reshape([6, 8, 5, 5]))
                                checkpoint1["module_list.3.bias"]=torch.tensor(np.array(x1[vec][1408:1414].cpu()).reshape([6]))
        
                                checkpoint1["module_list.6.weight"]=torch.tensor(np.array(x1[vec][1414:1510].cpu()).reshape([4, 6, 2, 2]))
                                checkpoint1["module_list.6.bias"]=torch.tensor(np.array(x1[vec][1510:1514].cpu()).reshape([4]))
        
                                checkpoint1["module_list.9.weight"]=torch.tensor(np.array(x1[vec][1514:2234].cpu()).reshape([20,36]))
                                checkpoint1["module_list.9.bias"]=torch.tensor(np.array(x1[vec][2234:2254].cpu()).reshape([20]))
        
                                checkpoint1["module_list.11.weight"]=torch.tensor(np.array(x1[vec][2254:2454].cpu()).reshape([10,20]))
                                checkpoint1["module_list.11.bias"]=torch.tensor(np.array(x1[vec][2454:2464].cpu()).reshape([10]))
                            
                                CNN_x1=copy.deepcopy(Brain)
                                CNN_x1.load_state_dict(checkpoint1)
                                # Extract weights from a specific layer (e.g., 'fc' layer)
                                # for name, param in CNN_x1.named_parameters():
                                #     if len(param.shape)>1:
                                #         weights = param
                                #         weights = weights.view(weights.shape[0], -1)
                                #         weights = weights.T @ weights
                                #         # Compute eigenvalues of the weight matrix
                                #         eigenvalues = torch.linalg.eigvals(weights.detach().cpu()).numpy()
    
                                #         # Extract real and imaginary parts
                                #         real_eigenvalues = eigenvalues.real
                                #         imag_eigenvalues = eigenvalues.imag
    
                                #             # Plot histogram of the real parts of the eigenvalues
                                #         plt.figure(figsize=(8, 6))
                                #         plt.hist(real_eigenvalues, bins=30, alpha=0.7, color='blue', edgecolor='black')
                                #         plt.xlabel('Real Part of Eigenvalues')
                                #         plt.ylabel('Frequency')
                                #         plt.title(f'Input 1 Eigenvalues of layer {name} shaped {tuple(param.shape)} model number {vec}')
                                        
                                #         plt.grid(True)
                                #         #plt.show()
                                #         image_path = results_path+f"Tracking/Input 1 Eigenvalues of layer {name} shaped {tuple(param.shape)} model number {vec}.png"
                                #         plt.savefig(image_path)
                                        
                                #         wandb.log({f"Input 1 Eigenvalues of layer {name} shaped {tuple(param.shape)} model number {vec}":  wandb.Image(image_path,caption=f"Input 1 Eigenvalues of layer {name} shaped {tuple(param.shape)} model number {vec}")})
                                #         plt.close()
                            
    
                                for name, param in CNN_x1.named_parameters():
                                    if len(param.shape) > 1:
                                        weights = param
                                        weights = weights.view(weights.shape[0], -1)
                                        weights = weights.T @ weights
                                        
                                        # Compute eigenvalues of the weight matrix
                                        eigenvalues = torch.linalg.eigvals(weights.detach().cpu()).numpy()
                                        
                                        # Extract real and imaginary parts
                                        real_eigenvalues = eigenvalues.real
                                        imag_eigenvalues = eigenvalues.imag
                                        
                                        # Log histogram to wandb
                                        wandb.log({
                                            f"Input 1 Eigenvalues of layer {name} shaped {tuple(param.shape)}": 
                                            wandb.Histogram(real_eigenvalues, num_bins=64)
                                        })
                                        
                                
                                checkpoint2["module_list.0.weight"]=torch.tensor(np.array(x2[vec][0:200].cpu()).reshape([8, 1, 5, 5]))
                                checkpoint2["module_list.0.bias"]=torch.tensor(np.array(x2[vec][200:208].cpu()).reshape([8]))
        
                                checkpoint2["module_list.3.weight"]=torch.tensor(np.array(x2[vec][208:1408].cpu()).reshape([6, 8, 5, 5]))
                                checkpoint2["module_list.3.bias"]=torch.tensor(np.array(x2[vec][1408:1414].cpu()).reshape([6]))
        
                                checkpoint2["module_list.6.weight"]=torch.tensor(np.array(x2[vec][1414:1510].cpu()).reshape([4, 6, 2, 2]))
                                checkpoint2["module_list.6.bias"]=torch.tensor(np.array(x2[vec][1510:1514].cpu()).reshape([4]))
        
                                checkpoint2["module_list.9.weight"]=torch.tensor(np.array(x2[vec][1514:2234].cpu()).reshape([20,36]))
                                checkpoint2["module_list.9.bias"]=torch.tensor(np.array(x2[vec][2234:2254].cpu()).reshape([20]))
        
                                checkpoint2["module_list.11.weight"]=torch.tensor(np.array(x2[vec][2254:2454].cpu()).reshape([10,20]))
                                checkpoint2["module_list.11.bias"]=torch.tensor(np.array(x2[vec][2454:2464].cpu()).reshape([10]))
                            
                                CNN_x2=copy.deepcopy(Brain)
                                CNN_x2.load_state_dict(checkpoint2)
                                # Extract weights from a specific layer (e.g., 'fc' layer)
                                # for name, param in CNN_x2.named_parameters():
                                #     if len(param.shape)>1:
                                #         weights = param
                                #         weights = weights.view(weights.shape[0], -1)
                                #         weights = weights.T @ weights
                                #         # Compute eigenvalues of the weight matrix
                                #         eigenvalues = torch.linalg.eigvals(weights.detach().cpu()).numpy()
    
                                #         # Extract real and imaginary parts
                                #         real_eigenvalues = eigenvalues.real
                                #         imag_eigenvalues = eigenvalues.imag
    
                                #             # Plot histogram of the real parts of the eigenvalues
                                #         plt.figure(figsize=(8, 6))
                                #         plt.hist(real_eigenvalues, bins=30, alpha=0.7, color='blue', edgecolor='black')
                                #         plt.xlabel('Real Part of Eigenvalues')
                                #         plt.ylabel('Frequency')
                                #         plt.title(f'Input 2 Eigenvalues of layer {name} shaped {tuple(param.shape)} model number {vec}')
                                        
                                #         plt.grid(True)
                                #         #plt.show()
                                #         image_path = results_path+f"Tracking/Input 2 Eigenvalues of layer {name} shaped {tuple(param.shape)} model number {vec}.png"
                                #         plt.savefig(image_path)
                                        
                                #         wandb.log({f"Input 2 Eigenvalues of layer {name} shaped {tuple(param.shape)} model number {vec}":  wandb.Image(image_path,caption=f"Input 2 Eigenvalues of layer {name} shaped {tuple(param.shape)} model number {vec}")})
                                #         plt.close()
                                
    
                                for name, param in CNN_x2.named_parameters():
                                    if len(param.shape) > 1:
                                        weights = param
                                        weights = weights.view(weights.shape[0], -1)
                                        weights = weights.T @ weights
                                        
                                        # Compute eigenvalues of the weight matrix
                                        eigenvalues = torch.linalg.eigvals(weights.detach().cpu()).numpy()
                                        
                                        # Extract real and imaginary parts
                                        real_eigenvalues = eigenvalues.real
                                        imag_eigenvalues = eigenvalues.imag
                                        
                                        # Log histogram to wandb
                                        wandb.log({
                                            f"Input 1 Eigenvalues of layer {name} shaped {tuple(param.shape)}": 
                                            wandb.Histogram(real_eigenvalues, num_bins=64)
                                        })
                                        
                               
                                for name, param in model.named_parameters():
                                    if len(param.shape) > 1:
                                        weights = param
                                        weights = weights.view(weights.shape[0], -1)
                                        weights = weights.T @ weights
                                        
                                        # Compute eigenvalues of the weight matrix
                                        eigenvalues = torch.linalg.eigvals(weights.detach().cpu()).numpy()
                                        
                                        # Extract real and imaginary parts
                                        real_eigenvalues = eigenvalues.real
                                        imag_eigenvalues = eigenvalues.imag
                                        
                                        # Log histogram to wandb
                                        wandb.log({
                                            f"Input 1 Eigenvalues of layer {name} shaped {tuple(param.shape)}": 
                                            wandb.Histogram(real_eigenvalues, num_bins=64)
                                        })
                                        
                                criterion_CNN0=CrossEntropyLoss()
        
                                test_IF0=ClassSpecificImageFolder( root="./data/SplitMnist/test/",dropped_classes=[str(x) for x in L2],transform=transforms.Compose([ transforms.ToTensor(),transforms.Grayscale(1)]))
                                Ts_DL0 = DataLoader(dataset=test_IF0, batch_size=36, num_workers=0, shuffle=False)
        
                                _, valid_epoch_acc0,correct_counts,total_counts= validate(model, Ts_DL0,  criterion_CNN0,10,epoch,vec)
    
                                if len(task3)==10:
                                    valid_epoch_acc1=valid_epoch_acc0
                                    continue
                                else:
                                    criterion_CNN1=CrossEntropyLoss()
                                    test_IF1=ClassSpecificImageFolder( root="./data/SplitMnist/test/",dropped_classes=[str(x) for x in task3],transform=transforms.Compose([ transforms.ToTensor(),transforms.Grayscale(1)]))
                                    Ts_DL1 = DataLoader(dataset=test_IF1, batch_size=36, num_workers=0, shuffle=False)
        
                                valid_epoch_loss0, valid_epoch_acc1,correct_counts,total_counts= validate(model, Ts_DL1,  criterion_CNN1,10,epoch,vec)
                                epoch_nb=-1
    
                                print("Reconstructed cnn acc ID",valid_epoch_acc0)
                                print("Reconstructed cnn acc OOD",valid_epoch_acc1)
                                DF.at[track,"Reconstructed Accuracy ID"]=valid_epoch_acc0
        
                                optimizerCNN = Adam(model.parameters(), lr=0.05)
                                schedulerCNN = torch.optim.lr_scheduler.CyclicLR(optimizerCNN ,base_lr=1e-3, max_lr=0.1, step_size_up=400, mode="triangular2", cycle_momentum=False)
                                criterion_CNN=CrossEntropyLoss()
        
        
                                train_IF0=ClassSpecificImageFolder( root="./data/SplitMnist/train/",dropped_classes=[str(x) for x in L2],transform=transforms.Compose([ transforms.ToTensor(),transforms.Grayscale(1)]))
                                Tr_DLr = DataLoader(dataset=train_IF0, batch_size=36, num_workers=0, shuffle=True)
        
        
                                fine_tune_needed=0
                                #FINETUNING
                                for epoch_cnn in range(epochs_fn):
                                    if epoch_cnn==0:
                                        train_epoch_loss, train_epoch_acc = train(model, Tr_DLr, optimizerCNN, criterion_CNN,10,df=DF,First=True)
                                        valid_epoch_loss0FN, valid_epoch_acc0FN,correct_counts,total_counts= validate(model, Ts_DL0,  criterion_CNN,10,epoch,vec)
                                        DF.at[track,"epoch 0"]=valid_epoch_acc0FN
                                    else:
                                        train_epoch_loss, train_epoch_acc = train(model, Tr_DLr, optimizerCNN, criterion_CNN,10,df=DF)
                                        valid_epoch_loss0FN, valid_epoch_acc0FN,correct_counts,total_counts= validate(model, Ts_DL0,  criterion_CNN,10,epoch,vec)
                                        
                                        DF.at[track,f"epoch {epoch_cnn}"]=valid_epoch_acc0FN
                                    schedulerCNN.step()
                                    fine_tune_needed+=1
        
                                for name, param in model.named_parameters():
                                    if len(param.shape) > 1:
                                        weights = param
                                        weights = weights.view(weights.shape[0], -1)
                                        weights = weights.T @ weights
                                        
                                        # Compute eigenvalues of the weight matrix
                                        eigenvalues = torch.linalg.eigvals(weights.detach().cpu()).numpy()
                                        
                                        # Extract real and imaginary parts
                                        real_eigenvalues = eigenvalues.real
                                        imag_eigenvalues = eigenvalues.imag
                                        
                                        # Log histogram to wandb
                                        wandb.log({
                                            f"Input 1 Eigenvalues of layer {name} shaped {tuple(param.shape)}": 
                                            wandb.Histogram(real_eigenvalues, num_bins=64)
                                        })
                                    
    
                            L_param=[]
                            for param in model.parameters():
                                m = nn.Flatten(0,-1)
                                L_param.append(m(param))
                            vec1FN = torch.Tensor()
                            for idx in L_param:
                                vec1FN = torch.cat((vec1FN, idx.view(-1)))
                            vec1FN=vec1FN.detach().cpu()
                            vec2=tg[vec].cpu()
                            DF.loc[track,Cols[4930:7394]]=vec1FN.tolist()
                            MSE=criterion(vec1FN,vec2).item()
                            MS1=criterion(vec1FN[:208],vec2[:208]).item()
                            MS2=criterion(vec1FN[208:1414],vec2[208:1414]).item()
                            MS3=criterion(vec1FN[1414:1514],vec2[1414:1514]).item()
                            MS4=criterion(vec1FN[1514:2254],vec2[1514:2254]).item()
                            MS5=criterion(vec1FN[2254:],vec2[2254:]).item()
    
    
                            #print("MSE :",MS1,MS2,MS3,MS4,MS5,MSE)
                            DF.at[track,"MSE FN"]=MSE
                            DF.at[track,"MSE 1 FN"]=MS1
                            DF.at[track,"MSE 2 FN"]=MS2
                            DF.at[track,"MSE 3 FN"]=MS3
                            DF.at[track,"MSE 4 FN"]=MS4
                            DF.at[track,"MSE 5 FN"]=MS5
                            kl_loss = nn.KLDivLoss(reduction="mean")
                            KLD=kl_loss(vec1FN,vec2).item() #pred, true
                            KL1=kl_loss(vec1FN[:208],vec2[:208]).item()
                            KL2=kl_loss(vec1FN[208:1414],vec2[208:1414]).item()
                            KL3=kl_loss(vec1FN[1414:1514],vec2[1414:1514]).item()
                            KL4=kl_loss(vec1FN[1514:2254],vec2[1514:2254]).item()
                            KL5=kl_loss(vec1FN[2254:],vec2[2254:]).item()
                            #print("KLD :",KLD,KL1,KL2,KL3,KL4,KL5)
                            DF.at[track,"KL divergence FN"]=KLD
                            DF.at[track,"KL 1 FN"]=KL1
                            DF.at[track,"KL 2 FN"]=KL2
                            DF.at[track,"KL 3 FN"]=KL3
                            DF.at[track,"KL 4 FN"]=KL4
                            DF.at[track,"KL 5 FN"]=KL5
    
                            WSD=wasserstein_distance(vec1FN,vec2)
                            WS1=wasserstein_distance(vec1FN[:208],vec2[:208])
                            WS2=wasserstein_distance(vec1FN[208:1414],vec2[208:1414])
                            WS3=wasserstein_distance(vec1FN[1414:1514],vec2[1414:1514])
                            WS4=wasserstein_distance(vec1FN[1514:2254],vec2[1514:2254])
                            WS5=wasserstein_distance(vec1FN[2254:],vec2[2254:])
                            #print("WSD :",WSD,WS1,WS2,WS3,WS4,WS5)
    
                            DF.at[track,"WSD FN"]=WSD
                            DF.at[track,"WS 1 FN"]=WS1
                            DF.at[track,"WS 2 FN"]=WS2
                            DF.at[track,"WS 3 FN"]=WS3
                            DF.at[track,"WS 4 FN"]=WS4
                            DF.at[track,"WS 5 FN"]=WS5
                            for name, param in mod.named_parameters():
                                if name == 'vec2neck.weight':
                                    W = param
                                    break
                            CL=loss_Contractive(mod.vec2neck.weight,vec1FN,vec2, output[1], 0.00001)        
                            #print("Contractive :",CL.cpu().item())
                            DF.at[track,"contractive distance FN"]=CL.cpu().item()
                            Norm1=np.sum(np.abs(vec1FN.numpy()))
                            N11=np.sum(np.abs(vec1FN[:208].numpy()))
                            N12=np.sum(np.abs(vec1FN[208:1414].numpy()))
                            N13=np.sum(np.abs(vec1FN[1414:1514].numpy()))
                            N14=np.sum(np.abs(vec1FN[1514:2254].numpy()))
                            N15=np.sum(np.abs(vec1FN[2254:].numpy()))
                            #print("Weight pred L1: ",Norm1, N11,N12,N13,N14,N15)
                            DF.at[track,"N1 FN"]=Norm1
                            DF.at[track,"N11 FN"]=N11
                            DF.at[track,"N12 FN"]=N12
                            DF.at[track,"N13 FN"]=N13
                            DF.at[track,"N14 FN"]=N14
                            DF.at[track,"N15 FN"]=N15
                            Norm2=np.sum(np.abs(vec2.numpy()))
                            N21=np.sum(np.abs(vec2[:208].numpy()))
                            N22=np.sum(np.abs(vec2[208:1414].numpy()))
                            N23=np.sum(np.abs(vec2[1414:1514].numpy()))
                            N24=np.sum(np.abs(vec2[1514:2254].numpy()))
                            N25=np.sum(np.abs(vec2[2254:].numpy()))
                            #print("Weight GT L1: ",Norm2, N21,N22,N23,N24,N25)
                            DF.at[track,"N2 FN"]=Norm2
                            DF.at[track,"N21 FN"]=N21
                            DF.at[track,"N22 FN"]=N22
                            DF.at[track,"N23 FN"]=N23
                            DF.at[track,"N24 FN"]=N24
                            DF.at[track,"N25 FN"]=N25
                            #print("Saturation in pred ",100*sum(1 for x in vec1FN if x > 0.95 or x<0.05)/len(vec1FN),"% \t saturaion in target",100*sum(1 for x in vec2 if x > 0.95 or x<0.05)/len(vec2),"%")
                            #print("LWLN ",LW(vec1FN,vec2).item())
                            DF.at[track,"saturated in pred FN(%)"]=100*sum(1 for x in vec1FN if x > 0.95 or x<0.05)/len(vec1FN)
                            DF.at[track,"saturated in GT FN(%)"]=100*sum(1 for x in vec2 if x > 0.95 or x<0.05)/len(vec2)
                            DF.at[track,"LWLN FN"]=LWs(vec1FN,vec2).cpu().item()
    
                            
                            track=track+1
                    
                    try :
                        print(f"{str(datetime.datetime.now())} begin topo")
                        # Define filter function – can be any scikit-learn transformer
                        filter_func = Projection(columns=[0,1])
                        # Define cover
                        cover = CubicalCover(n_intervals=12, overlap_frac=0.15)
                        # Choose clustering algorithm – default is DBSCAN
                        clusterer = DBSCAN()
        
                        # Configure parallelism of clustering step
                        n_jobs = 8
        
                        GT=DF[Cols[2:2466]]
                        dataG=GT.to_numpy()
                        #print(Cols[2],Cols[2466],Cols[4930])
                        PD=DF[Cols[2466:4930]]
                        dataP=PD.to_numpy()
        
                        FN=DF[Cols[4930:7394]]
                        dataF=FN.to_numpy()
                        # Initialise pipeline
                        pipe = make_mapper_pipeline(filter_func=filter_func,cover=cover,clusterer=clusterer,verbose=False,n_jobs=n_jobs)
        
        
                        plotly_params = {"node_trace": {"marker_colorscale": "Blues"}}
                        fig = plot_static_mapper_graph(pipe, dataG, color_data=dataG, plotly_params=plotly_params)
        
                        fig1 = go.Figure(fig)
                        #fig1.write_image(results_path+f"Tracking/Mapper GT Graph 10 test set batches {epoch}.png")
                        wandb.log({f"Mapper GTruth Graph 10 test set {nb_test} batches ": wandb.Plotly(fig1)})
        
        
        
                        fig = plot_static_mapper_graph(pipe, dataP, color_data=dataP, plotly_params=plotly_params)
        
                        fig1 = go.Figure(fig)
                        #fig1.write_image(results_path+f"Tracking/Mapper Pred Graph 10 test set batches {epoch}.png")
                        wandb.log({f"Mapper Pred Graph 10 test set {nb_test} batches ": wandb.Plotly(fig1)})
        
        
                        fig = plot_static_mapper_graph(pipe, dataF, color_data=dataF, plotly_params=plotly_params)
        
                        fig1 = go.Figure(fig)
                        #fig1.write_image(results_path+f"Tracking/Mapper FN Graph 10 test set batches {epoch}.png")
                        wandb.log({f"Mapper Finetuned Graph test set {nb_test} batches ": wandb.Plotly(fig1)})
                        print(f"{str(datetime.datetime.now())} end topo")
                    except Exception as e:
                        print(f"{str(datetime.datetime.now())} {e}")
                    wandb.log({"Epoch 0 Test CNN accuracy on 10 batches ": DF['epoch 0'].mean()})
                    wandb.log({"Epoch 1 Test CNN accuracy on 10 batches ": DF['epoch 1'].mean()})
                    wandb.log({"Epoch 2 Test CNN accuracy on 10 batches ": DF['epoch 2'].mean()})
                    wandb.log({"Epoch 3 Test CNN accuracy on 10 batches ": DF['epoch 3'].mean()})
                    wandb.log({"Epoch 4 Test CNN accuracy on 10 batches ": DF['epoch 4'].mean()})
    
    
    
                if (epoch+1)%50==0:
                    DF.to_csv(results_path+f"Tracking/AE epoch {combination} {epoch}.csv")
                    torch.save({'epoch':epoch,'model_state_dict': mod.state_dict(),
                                'optimizerENC1_state_dict':  optimizerEnc1.state_dict() ,
                                'optimizerENC2_state_dict':optimizerEnc2.state_dict(),
                                'optimizerDense_state_dict':optimizerDense.state_dict(),
                                'optimizerDec_state_dict': optimizerDec.state_dict(),
                                'Batch Loss':used_Loss.detach().cpu().item(),
                                'validation entire testset MSE':loss_value.detach().cpu().item()},
                                results_path+'AE epoch {} {} {}.pth'.format(combination,track,epoch))
                    torch.cuda.empty_cache()
                if (epoch+1)==num_epochs:
                    del(mod)
                    del(optimizerEnc1)
                    del(optimizerEnc2) 
                    del(optimizerDense)
                    del(optimizerDec)
                    torch.cuda.empty_cache()
                    torch.cuda.synchronize()
                    gc.collect()
                
            wandb.finish()
            track=+1
        #     return loss_tr
        # study= optuna.create_study(direction="minimize",storage=storage)
        # study.optimize(objective,n_trials=2,callbacks=[lambda study, trial: gc.collect()]+[logging_callback])

./data/Scenario/overlapping 3 [0, 1, 2, 3, 4, 5] 26 silu/train pair.npy 450
./data/Scenario/overlapping 3 [0, 1, 2, 3, 4, 5] 11 gelu/train pair.npy 440
./data/Scenario/overlapping 4 [0, 1, 2, 3, 4, 5] 11 relu/train pair.npy 450
./data/Scenario/overlapping 2 [0, 1, 2, 3, 4, 5] 21 sigmoid/train pair.npy 450
./data/Scenario/overlapping 1 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 31 gelu/train pair.npy 55572
./data/Scenario/overlapping 0 [0, 1, 2, 3, 4, 5] 16 gelu/train pair.npy 20
./data/Scenario/overlapping 4 [0, 1, 2, 3, 4, 5] 16 leakyrelu/train pair.npy 450
./data/Scenario/overlapping 4 [0, 1, 2, 3, 4, 5] 36 silu/train pair.npy 450
./data/Scenario/overlapping 1 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 16 leakyrelu/train pair.npy 55572
./data/Scenario/overlapping 3 [0, 1, 2, 3, 4, 5] 16 tanh/train pair.npy 450
./data/Scenario/overlapping 0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 26 relu/train pair.npy 55572
./data/Scenario/overlapping 0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 36 tanh/train pair.npy 55572
./data/Scenario/over

CommError: Run initialization has timed out after 90.0 sec. Please try increasing the timeout with the `init_timeout` setting: `wandb.init(settings=wandb.Settings(init_timeout=120))`.

In [ ]:
d_loos["JS Loss"]

In [ ]:
print(f"Type of used_Loss: {type(used_Loss)}")
print(f"Is tensor: {torch.is_tensor(used_Loss)}")
if torch.is_tensor(used_Loss):
    print(f"Requires grad: {used_Loss.requires_grad}")

accelerator.backward(used_Loss)

In [ ]:
DF.shape , j

In [ ]:
# Get indices of NaN values
nan_indices = np.where(np.isnan(dataP))[0]
print(nan_indices)  # [2 4]

# Delete those indices
dataP = np.delete(dataP, nan_indices)
print(dataP.shape)  # [1. 2. 4. 6.]

In [ ]:
GT.describe()

In [ ]:
FN.describe()

In [ ]:
vec1FN

In [ ]:
filter_func = Projection(columns=[0,1])
# Define cover
cover = CubicalCover(n_intervals=12, overlap_frac=0.5)
# Choose clustering algorithm – default is DBSCAN
clusterer = DBSCAN()

# Configure parallelism of clustering step
n_jobs = 8
dataG = np.random.random((30, 30))
pipe = make_mapper_pipeline(filter_func=filter_func,cover=cover,clusterer=clusterer,verbose=False,n_jobs=n_jobs)


plotly_params = {"node_trace": {"marker_colorscale": "Blues"}}
fig = plot_static_mapper_graph(pipe, dataG, color_data=dataG, plotly_params=plotly_params)

fig1 = go.Figure(fig)
#fig1.write_image(results_path+f"Tracking/Mapper GT Graph 10 test set batches {epoch}.png")
wandb.log({"Mapper GT Graph 10 test set batches": wandb.Image(fig1)


fig1

In [ ]:
#!pip install plotly==5.11.0
!pip install -U kaleido

In [ ]:
Eigenvalues
profiler
Multipers
Bottleneck distance
Optimal transport mapping imagery
Shapley
Add 3 dicts instead of 1 to detect significant relative change and launch Reconstructions/checkpointing
Kolmagrov-Arnold functions
Matrix diagonalization github
RL
Gromov-wassertstein

# Testing Loop

In [ ]:
\usepackage{amsmath}
\usepackage{algorithmic}
\usepackage{algpseudocode}
\usepackage{algorithm}

